In [22]:
import pandas as pd
import numpy as np

In [24]:
dataset=pd.ExcelFile("C:/Users/NagarjunaPujala/Downloads/python_hackathon/V02Max_Combined.xlsx")

In [25]:
Athletes=pd.read_excel(dataset, sheet_name='Athletes')
Kayaking=pd.read_excel(dataset, sheet_name='Kayaking')
Triathlon=pd.read_excel(dataset, sheet_name='Triathlon')
Fencing=pd.read_excel(dataset, sheet_name='Fencing')

C:\Users\NagarjunaPujala\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\NagarjunaPujala\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():
C:\Users\NagarjunaPujala\anaconda3\Lib\site-packages\openpyxl\worksheet\_read_only.py:81: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [27]:
# Add a new column to indicate the sport category
Kayaking['sport'] = 'Kayaking'
Triathlon['sport'] = 'Triathlon'
Fencing['sport'] = 'Fencing'
# Concatenate the DataFrames
All_sports=pd.concat([Kayaking, Triathlon, Fencing], ignore_index=True)
#All_sports.head(40)
# Set 'sport' as the index
All_sports.set_index(['sport'], inplace=True)

In [9]:
print(All_sports)

          ID     time     RR       VO2  power
sport                                        
Kayaking  11 -168.600  727.0  0.287147      0
Kayaking  11 -167.872  728.0  0.287147      0
Kayaking  11 -167.148  724.0  0.287147      0
Kayaking  11 -166.484  664.0  0.320580      0
Kayaking  11 -165.836  648.0  0.320580      0
...       ..      ...    ...       ...    ...
Fencing   10  942.700  428.0  0.786363      0
Fencing   10  943.124  424.0  0.786363      0
Fencing   10  943.552  428.0  0.786363      0
Fencing   10  943.976  424.0  0.786363      0
Fencing   10  944.404  428.0  0.786363      0

[52062 rows x 5 columns]


### 65.Query:
   [ 1,  2,  3,  4,  5],
   [ 6,  7,  8,  9, 10],
   [11, 12, 13, 14, 15],
   [16, 17, 18, 19, 20],
   [21, 22, 23, 24, 25],
   [26, 27, 28, 29, 30]  Use array slicing to display elements 4,7,10,17.

In [14]:
# solution:
fived_array=np.array([ [ 1,  2,  3,  4,  5],
   [ 6,  7,  8,  9, 10],
   [11, 12, 13, 14, 15],
   [16, 17, 18, 19, 20],
   [21, 22, 23, 24, 25],
   [26, 27, 28, 29, 30] ])
print(fived_array[0][3],",",fived_array[1][1],",",fived_array[1][4],",",fived_array[3][1])

4 , 7 , 10 , 17


### 66.Query: On average, how long did each participant spend at 200 watts?

In [18]:
#solution:
rslt_df = All_sports.loc[All_sports['power'] == 200]
rslt = rslt_df.groupby(['ID'])
result=rslt['time'].max()-rslt['time'].min()
A=len(pd.unique(All_sports['ID']))
print(result/A)

ID
3     3.247333
4     3.313333
5     3.282667
6     3.278444
9     0.301111
10    1.753778
11    3.312000
12    3.254889
13    3.374667
14    3.315778
15    3.319333
16    3.314444
18    1.465333
Name: time, dtype: float64


### 67.Query:Athletes from which sport had the highest average cardiorespiratory fitness in terms of v02 max?

In [30]:
#solution:
sum_vo2=All_sports.groupby(['sport','ID'])['VO2'].sum()
df1=pd.merge(sum_vo2,Athletes,on='ID',how='inner')[['ID','VO2','sport','Weight']]
for index,row in df1.iterrows():
      df1.at[index,'max_vo2']=round(df1.at[index,'VO2']/df1.at[index,'Weight'],2)
      df2=df1.groupby('sport')['max_vo2'].mean()
print(df2.idxmax(),"sport had the highest average cardiorespiratory fitness in terms of vo2 max with value:",df2.max())

triathlon sport had the highest average cardiorespiratory fitness in terms of vo2 max with value: 77.5


### 68.Query: What % of athletes exceeded 90% of max HR? (Max HR is calculated as 220-Age)


In [37]:
#solution:
All_sports['hr_cal']= (60000/All_sports['RR']).round(2)
R_11= All_sports.groupby(['sport','ID'])['hr_cal'].max()
df1=pd.merge(R_11,Athletes,on='ID',how='inner')[['ID','hr_cal','sport','age']]
for index,row in All_sports.iterrows():
    df1['max_HR']= abs(df1['age'].subtract(220))
df1.loc[df1['hr_cal']>0.9*df1['max_HR'],"solution"]="Exceeds 90%"
df1.loc[df1['hr_cal']<=0.9*df1['max_HR'],"solution"]="not Exceeds 90%"
print(df1['solution'].value_counts(normalize=True).mul(100).round(2).astype(str)+'%')

solution
Exceeds 90%        94.44%
not Exceeds 90%     5.56%
Name: proportion, dtype: object


### 69.Query:What was the lowest heart rate recorded at a power wattage of 200


In [42]:
#solution:
All_sports['lowest HR'] = 60000/All_sports['RR']
rslt_df_2 = All_sports.loc[All_sports['power'] == 200]
print(rslt_df_2[['ID','power','lowest HR']].min())

ID             3.000000
power        200.000000
lowest HR     71.770335
dtype: float64


### 70.Query: On Average how long did most athletes spend at 75% of MaxHR (Max HR is calculated as 220-Age)


In [45]:
#solution:
Athletes['Max_HR']=220-Athletes['age']
Athletes['75%Max_HR']=round(Athletes['Max_HR']*0.75,0)
All_sports['HR']=round(60000/All_sports['RR'],0)
df1=pd.merge(All_sports,Athletes,on='ID',how='inner')[['ID','75%Max_HR','time','HR']]
df2=df1.loc[df1['75%Max_HR']==df1['HR']].reset_index()
df3=df2.groupby('ID')['time'].max()-df2.groupby('ID')['time'].min()
df3.mean()

515.1886666666667

### 71.Query: Create a crosstab to show each sport as a column and each heart rate zone as rows with counts of rows that satisfy each condition as values

In [51]:
#solution:
All_sports['hr_cal']= (60/(All_sports['RR']/1000)).round(0)
df1=pd.merge(All_sports,Athletes,on='ID',how='inner')[['ID','hr_cal','sport','time','age']]
for index,row in All_sports.iterrows():
    df1['max_HR']= abs(df1['age'].subtract(220))
df1.loc[(df1['hr_cal']<=0.5*df1['max_HR']),"HR ZONE"]="ZONE 0"    
df1.loc[(df1['hr_cal']>=0.5*df1['max_HR']) & (df1['hr_cal']<=0.6*df1['max_HR']),"HR ZONE"]="ZONE 1"
df1.loc[(df1['hr_cal']>=0.6*df1['max_HR']) & (df1['hr_cal']<=0.7*df1['max_HR']),"HR ZONE"]="ZONE 2"
df1.loc[(df1['hr_cal']>=0.7*df1['max_HR']) & (df1['hr_cal']<=0.8*df1['max_HR']),"HR ZONE"]="ZONE 3"
df1.loc[(df1['hr_cal']>=0.8*df1['max_HR']) & (df1['hr_cal']<=0.9*df1['max_HR']),"HR ZONE"]="ZONE 4"
df1.loc[(df1['hr_cal']>=0.9*df1['max_HR']) & (df1['hr_cal']<=df1['max_HR']),"HR ZONE"]="ZONE 5"
result = pd.crosstab(df1['sport'], df1['HR ZONE'])
print(result)

HR ZONE    ZONE 0  ZONE 1  ZONE 2  ZONE 3  ZONE 4  ZONE 5
sport                                                    
fencing      2380    4097    5839    5848    7209    2225
kayak        1517    3265    4594    3867    4365    1157
triathlon     473     479    1409     871    1101     212


### 72.Query: Provide an example of a lambda function that multiplies any 2 input values.


In [54]:
#solution:
multi=lambda X,Y:X*Y
X = float(input('Enter X value: '))
Y = float(input('Enter Y value: '))
result = multi(X, Y)
print("The multipliaction of the X & Y values:", result)

Enter X value:  4
Enter Y value:  5


The multipliaction of the X & Y values: 20.0


### 73.Query:How long was athlete 17s warm up


In [62]:
#solution:
df1=All_sports.loc[(All_sports['time']<=0)&(All_sports['ID']==17)].reset_index()
df2=df1['time'].max()-df1['time'].min()
print(df2,"Seconds long was athlete 17s warm up")

182.408 Seconds long was athlete 17s warm up


### 74 Query: Take a students full name and reverse it using a for loop

In [109]:
#solution:
Athletes['full_name'] = Athletes['First Name'] + ' ' + Athletes['Last Name']
reverse_list=[]
for names in Athletes['full_name']:
    reverse_name=""
    for char in names:
         reverse_name= char + reverse_name
    reverse_list.append(reverse_name)
Athletes['reverse']=reverse_list     
print(Athletes[['ID','full_name','reverse']])     

    ID         full_name           reverse
0    1   Olivia Martinez   zenitraM aivilO
1    2    Ethan Reynolds    sdlonyeR nahtE
2    3    Jackson Harper    repraH noskcaJ
3    4      Liam Sanchez      zehcnaS maiL
4    5      Ava Thompson      nospmohT avA
5    6  Sophia Rodriguez  zeugirdoR aihpoS
6    7     Grace Simmons     snommiS ecarG
7    8     Aiden Bennett     ttenneB nediA
8    9       Chloe Price       ecirP eolhC
9   10       Lily Jordan       nadroJ yliL
10  11  Isabella Griffin  niffirG allebasI
11  12       Caleb Dixon       noxiD belaC
12  13     Mia Carpenter     retnepraC aiM
13  14   Harper Mitchell   llehctiM repraH
14  15     Mason Wallace     ecallaW nosaM
15  16    Logan Anderson    nosrednA nagoL
16  17      Lucas Foster      retsoF sacuL
17  18        Owen Myers        sreyM newO


### 75 Query:What was the maximum heart rate recorded during the test. Which athlete did it belong to? Display name, sport and HR

In [69]:
#solution:
Athletes['full_name'] = Athletes['First Name'] + ' ' + Athletes['Last Name']
All_sports['HR']= (60/(All_sports['RR']/1000))
df1=pd.merge(All_sports,Athletes,on='ID',how='inner')[['ID','full_name','sport','HR']]
df2=df1.loc[(df1['HR']==df1['HR'].max())].reset_index()
print(df2)


   index  ID      full_name    sport          HR
0  44041   7  Grace Simmons  fencing  238.095238


#### 76 Query: np.arange(1,31).reshape(6,5),find array slicing to get the elements 2,5,13,25,30.

In [101]:
#solution:
x= np.arange(1,31).reshape(6,5)
print(x)
print(x[0][1],",",x[0][4],",",x[2][2],",",x[4][4],",",x[5][4])

[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]
 [16 17 18 19 20]
 [21 22 23 24 25]
 [26 27 28 29 30]]
2 , 5 , 13 , 25 , 30


In [75]:
### 77 Query:

In [77]:
#solution:

In [79]:
### 78 Query:

In [81]:
#solution:

In [83]:
###79 Query:

In [85]:
#solution:

##### 80 Query:find the size of a Tuple for any given input

In [97]:
#solution:(80 Q)
#for integer input
result_tuple = tuple(int(item) for item in (input('Enter integers: ')).split())
print(result_tuple)
print(len(result_tuple),"is the tuple length of given input" )

Enter integers:  1 2 3 5 6 7 8


(1, 2, 3, 5, 6, 7, 8)
7 is the tuple length of given input


In [93]:
# for any input(80Q)
result_tuple = tuple((input('Enter integers: ')).split())
print(result_tuple)
print(len(result_tuple),"is the tuple length of given input" )

Enter integers:  q 2  4 4.5 -5 *


('q', '2', '4', '4.5', '-5', '*')
6 is the tuple length of given input
